In [1]:
import os

In [13]:
%pwd

'C:\\Users\\Ukant'

In [3]:
os.chdir("../")
%pwd

'd:\\college_work\\4th year\\Sem7th\\Project\\OBE\\Paper- Blooms taxanomy\\Boolm-s-Level-Detection-A-MLOPS-Project'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path 
    source_URL: str 
    unzip_dir: Path 
    local_data_path: Path    

In [5]:
from blooms.constants import *
from blooms.utils.common import read_yaml,create_directories 

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FIEL_PATH,
        schema_filepath = SCHEMA_FIEL_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion ## fetching the ele from config.yaml
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            unzip_dir=config.unzip_dir,
            local_data_path=config.local_data_path
        )
        return data_ingestion_config

In [7]:
import requests
import urllib.request as request
import zipfile
from blooms import logger
from blooms.utils.common import get_size

In [8]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_path):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_path
            )
            logger.info(f"{filename}, download! with the following info {headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_path))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-12-14 15:01:29: INFO: common: yaml file: config\config.yaml loaded succcessfully]
[2023-12-14 15:01:29: INFO: common: yaml file: params.yaml loaded succcessfully]
[2023-12-14 15:01:29: INFO: common: yaml file: schema.yaml loaded succcessfully]
[2023-12-14 15:01:29: INFO: common: created directory at : artifacts]
[2023-12-14 15:01:29: INFO: common: created directory at : artifacts/data_ingestion]
[2023-12-14 15:01:31: INFO: 2896558699: artifacts/data_ingestion/data.zip, download! with the following info Server: nginx
Date: Thu, 14 Dec 2023 09:33:05 GMT
Content-Type: application/zip
Content-Length: 15506
Last-Modified: Thu, 14 Dec 2023 07:57:24 GMT
Connection: close
ETag: "657ab564-3c92"
Strict-Transport-Security: max-age=63072000; includeSubDomains; preload
X-Frame-Options: sameorigin
X-Content-Type-Options: nosniff
X-XSS-Protection: 1; mode=block
Content-Security-Policy: default-src 'none'; media-src 'self'; style-src 'none' 'unsafe-inline'; img-src 'self' https://img.shields.io
